# Loading Libraries

In [1]:
import pydub
import os
import re
import numpy as np
import pandas as pd
import tensorflow as tf
from scipy.io.wavfile import read, write

C:\Users\HakiM\AppData\Local\Programs\Python\Python36\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


# Loading the dataset

In [ ]:
#training dataset
train_path  = '../dataset/wavLITE/train/'
errors = []
complete = 0
dataset = []
labels = []
for filename in os.listdir(train_path+'true/'):
    try:
        fs, y = read(train_path+'true/'+filename)
        dataset.append(np.array(y[5000000:5100000]))
        labels.append(1)
        complete = complete + 1
        if len(y) < 10000000:
            print(filename)
    except:
        errors.append(filename)
    print('Complete: ',(complete*100)/len(os.listdir(train_path+'true/')),'%')

complete = 0
for filename in os.listdir(train_path+'false/'):
    try:
        fs, y = read(train_path+'false/'+filename)
        dataset.append(np.array(y[5000000:5100000]))
        labels.append(1)
        complete = complete + 1
        if len(y) < 10000000:
            print(filename)
    except:
        errors.append(filename)
    print('Complete: ',(complete*100)/len(os.listdir(train_path+'false/')),'%')

print("Script complete with ", len(errors), " errors.")
if len(errors):
    for error in errors:
        print(error)

In [ ]:
#test dataset
test_data = []
test_labels = []
test_path  = '../dataset/wavLITE/test/'

complete = 0
for filename in os.listdir(test_path+'true/'):
    try:
        fs, y = read(test_path+'true/'+filename)
        test_data.append(np.array(y[5000000:5100000]))
        test_labels.append(1)
        complete = complete + 1
    except:
        errors.append(filename)
    print('Complete: ',(complete*100)/len(os.listdir(test_path+'true/')),'%')

complete = 0
for filename in os.listdir(test_path+'false/'):
    try:
        fs, y = read(test_path+'false/'+filename)
        test_data.append(np.array(y[5000000:5100000]))
        test_labels.append(0)
        complete = complete + 1
    except:
        errors.append(filename)
    print('Complete: ',(complete*100)/len(os.listdir(test_path+'false/')),'%')

print("Script complete with ", len(errors), " errors.")
if len(errors):
    for error in errors:
        print(error)

In [ ]:
#Convert dataset list of numpy arrays to list of tensors
for i in range(len(dataset)):    
    dataset[i] = tf.convert_to_tensor(dataset[i])
    labels[i] = tf.convert_to_tensor(dataset[i])

for i in range(len(test_data)):    
    test_data[i] = tf.convert_to_tensor(test_data[i])
    test_labels[i] = tf.convert_to_tensor(dataset[i])



In [ ]:
for i in range(len(test_data)):
    if len(test_data[i]) != len(test_data[i]):
        print(len(test_data[i]) , len(test_data[i]))

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((dataset, labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_data, test_labels))

In [ ]:
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 100

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

In [ ]:
print(train_dataset)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(1000000, 2)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10)
])

model.compile(optimizer=tf.keras.optimizers.RMSprop(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['sparse_categorical_accuracy'])

In [ ]:
model.fit(train_dataset, epochs=10)